In [2]:
import pathlib
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from torch import nn
import tqdm

## Get Data Loaders

In [33]:
# Create a pytorch dataset
data_dir = pathlib.Path('/data/tiny-imagenet/')
image_count = len(list(data_dir.glob('**/*.JPEG')))
CLASS_NAMES = np.array([item.name for item in (data_dir / 'train').glob('*')])
print('Discovered {} images'.format(image_count))

# Create the training data generator
batch_size = 32
im_height = 64
im_width = 64
num_epochs = 30

# data_transforms = transforms.Compose([
#     transforms.ToTensor(),
#     # transforms.Normalize((0, 0, 0), tuple(np.sqrt((255, 255, 255)))),
# ])

data_transforms = transforms.Compose([
    transforms.RandomCrop(64, padding=8),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
train_set = torchvision.datasets.ImageFolder(data_dir / 'train', data_transforms)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                           shuffle=True, num_workers=4, pin_memory=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Discovered 120000 images


## Model

In [85]:
model = torchvision.models.resnet50(pretrained=True)
checkpoint = torch.load('checkpoint.pth.tar')
model = nn.DataParallel(model)
model.load_state_dict(checkpoint['state_dict'])
model = model.module

In [86]:
# Create a simple model
for param in list(model.parameters())[:-30]:
    param.requires_grad = False
    
# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model.fc.in_features

model.fc =  nn.Sequential(
                  nn.Linear(num_ftrs, 256), 
                  nn.ReLU(),
                  nn.Linear(256, 256),
                  nn.ReLU(),
                  nn.Linear(256, 200))
model = model.to(device)
# model = Net(len(CLASS_NAMES), im_height, im_width)
optim = torch.optim.Adam(
    [
        {"params": list(model.parameters())[-30:-6], "lr": 1e-4},
        {"params": model.fc.parameters(), "lr": 1e-3}
    ])
criterion = nn.CrossEntropyLoss()

In [ ]:

for i in range(num_epochs):
    train_total, train_correct = 0,0
    for idx, (inputs, targets) in enumerate(train_loader):
        # print(inputs[1])
        # plt.imshow(inputs[1].permute(1,2,0))
        # plt.show()
        inputs, targets = inputs.to(device), targets.to(device)
        optim.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optim.step()
        _, predicted = outputs.max(1)
        train_total += targets.size(0)
        train_correct += predicted.eq(targets).sum().item()
        # if idx % 100 == 0:
        print("\r", end='')
        print(f'training {100 * idx / len(train_loader):.2f}%: {train_correct / train_total:.3f}', end='')
    torch.save({
        'net': model.state_dict(),
    }, 'resnet50-pretrainedaugmix-20epochs-nofreeze-aug.pt')


##test #1:0.038 at 5.06%


training 54.69%: 0.607